In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webelement import WebElement
from webdriver_manager.chrome import ChromeDriverManager
from typing import List, Tuple, Generator

In [2]:
# Instantiate a Chrome driver.
def load_driver() -> None:
    # Load and configure webdriver.
    options: Options = Options()
    # Stop browser windows from actually popping up.
    options.add_argument('--headless')
    # Install a browser for use by Selenium.
    service: Service = Service(executable_path=ChromeDriverManager().install())
    return Chrome(service=service, options=options)

In [3]:
# Helper function that navigates to the search result page for a particular
# month on the USCF page for historical tournament data.
def navigate_to_uscf_page(driver: Chrome, date_to_visit: str) -> None:
    # Navigate to US Chess tournament search page.
    USCF_URL = 'http://www.uschess.org/datapage/events-rated.php'
    driver.get(USCF_URL)
    date_search_box: WebElement = driver.find_element('name', 'month')
    date_search_box.clear()
    date_search_box.send_keys(date_to_visit)

    # Select CA as the State Code, which is where all 
    # chess.com USCF tournaments are registered.
    state_search_box: WebElement = driver.find_element('name', 'states')
    state_search_box.clear()
    state_search_box.send_keys('CA')
    state_search_box.submit()

In [4]:
# Scrapes tournment urls for the month that the driver is currently
# pointed add. Helper function navigate_to_uscf_page() navigates to the
# correct page.
def scrape_uscf_tournament_urls(driver: Chrome) -> List[str]:
    table_body: List[WebElement] = driver.find_elements(By.TAG_NAME, 'tbody')[2]
    table_body_row: List[WebElement] = table_body.find_elements(By.TAG_NAME, 'tr')
    url_list: List[str] = []
    for row in table_body_row:
        # Each row is a WebElement with data about one tournament.
        table_row: List[WebElement] = row.find_elements(By.TAG_NAME, 'td')
        if len(table_row) >= 3:
            url: str = None
            for element in table_row:
                if element.text.isnumeric() and len(element.text) > 10:
                    url = element.find_element(By.TAG_NAME, 'a').get_attribute('href')
                # Only keep tournament urls labeled "CHESS.COM"
                if 'CHESS.COM' in element.text.upper():
                    url_list.append(url)
    return url_list     

In [5]:
import sqlite3
from sqlite3 import Connection, Cursor

# Drop then create tables for all data scraped by this module.
def init_db() -> None:
    conn: Connection = sqlite3.Connection('scrape_data.db')
    cur: Cursor = conn.cursor()
    # # Stores urls to allow us to navigate to all relevant USCF tournaments.
    # cur.execute("""CREATE TABLE IF NOT EXISTS uscf_urls (
    #     id INTEGER PRIMARY KEY,
    #     date TEXT,
    #     url TEXT,
    #     scraped INTEGER
    #     )""")
    # cur.execute("""CREATE TABLE IF NOT EXISTS uscf_tournaments (
    #     id INTEGER PRIMARY KEY,
    #     tournament_name TEXT,
    #     tournament_code INTEGER,
    #     event_date TEXT, 
    #     received_date TEXT,
    #     entered_date TEXT, 
    #     rated_date TEXT,
    #     section_count INTEGER,
    #     player_count INTEGER,
    #     k_factor TEXT,
    #     rating_system TEXT,
    #     tournament_type TEXT,
    #     time_control TEXT,
    #     urls_id INTEGER,
    #     FOREIGN KEY (urls_id)
    #         REFERENCES uscf_urls (id)
    # )""")
    # cur.execute("""CREATE TABLE IF NOT EXISTS uscf_player_observations (
    #     id INTEGER PRIMARY KEY,
    #     name TEXT,
    #     seed_number INTEGER,
    #     url TEXT,
    #     uscf_id INTEGER,
    #     record TEXT,
    #     state_code TEXT,
    #     rating_type TEXT,
    #     before_rating TEXT,
    #     after_rating TEXT,
    #     color_assignments TEXT,
    #     uscf_tournaments_id INTEGER,
    #     FOREIGN KEY (uscf_tournaments_id)
    #         REFERENCES uscf_tournaments (id) 
    # )""")
    # cur.execute("""CREATE TABLE IF NOT EXISTS uscf_rounds (
    #     id INTEGER PRIMARY KEY,
    #     round_number INTEGER,
    #     result TEXT,
    #     opponent INTEGER,
    #     uscf_tournaments_id INTEGER,
    #     uscf_player_id INTEGER,
    #     FOREIGN KEY (uscf_tournaments_id)
    #         REFERENCES uscf_tournaments (id),
    #     FOREIGN KEY (uscf_player_id)
    #         REFERENCES uscf_player_observations (id)
    # )""")
    conn.commit()

In [6]:
# Scrapes a list of urls. Each url is a USCF tournament with a parallel entry
# on chess.com. Every month between 2015 and 2023 inclusive is checked.
def scrape_all_uscf_urls(driver: Chrome, cur: Cursor) -> List[str]:
    url_list: List[str] = []
    for year in range(2023, 2014, -1):
        # Page requires single digit months to have a 0 in front.
        for month in range(1, 10):
            date: str = '0' + str(month) + '/' + str(year)
            navigate_to_uscf_page(driver, date)
            url_list.extend(scrape_uscf_tournament_urls(driver))
        for month in range(10, 13):
            date: str = str(month) + '/' + str(year)
            navigate_to_uscf_page(driver, date)
            url_list.extend(scrape_uscf_tournament_urls(driver))
    return url_list

In [7]:
# Here is the code that scrapes the initial set of
# tournament URLs.

# driver: Chrome = load_driver()
# url_list: List[str] = scrape_all_uscf_urls(driver)

# url_tuples: List[Tuple] = []
# for url in url_list:
#     url_tuples.append((None, url, 0))

# conn: Connection = sqlite3.Connection('scrape_data.db')
# cur: Cursor = conn.cursor()
# cur.executemany("""INSERT INTO uscf_urls (
#     date, url, scraped
#     ) VALUES (?, ?, ? )""", url_tuples)
# conn.commit()

In [8]:
from bs4 import BeautifulSoup
from bs4.element import ResultSet, Tag
import requests
from requests.models import Response

# Currently set to only return data from one tourament page.
def get_tournament_page(tournament_id: str) -> BeautifulSoup:
    cur: Cursor = sqlite3.Connection('scrape_data.db').cursor()
    cur.execute("SELECT url FROM uscf_urls WHERE id = ?", (tournament_id,))
    url: str = cur.fetchall()[0][0]
    request: Response = requests.get(url)
    print(url)
    print('Status Code:', request.status_code)
    return BeautifulSoup(request.text, 'html.parser')

In [9]:
# Extracts the following in order: tournament_name, tournament_id, event_date, received_date,
# entered_date, rated_date, section_count, player_count
def extract_tournament_info1(soup: BeautifulSoup) -> List[str]:
    upper_table: ResultSet[Tag] = soup.find_all('table', attrs={
        'border': '0',
        'bgcolor': 'FFFFFF',
        'cellpadding': '3',
        'cellspacing': '0'
    })
    rows: ResultSet[Tag] = upper_table[0].find_all('tr')
    row1_tags: ResultSet[Tag] = rows[0].find_all('td')

    infoset1: List[str] = []
    infoset1.append(row1_tags[3].b.text)
    infoset1.append(row1_tags[3].small.text[1:-1])
    infoset1.append(row1_tags[7].b.text)

    dates_split: List[str] = row1_tags[13].b.text.split(' ')
    infoset1.append(dates_split[1])
    infoset1.append(dates_split[4])
    infoset1.append(dates_split[7])
    infoset1.append(row1_tags[15].b.text.split(' ')[0])
    infoset1.append(row1_tags[15].b.text.split(' ')[-2])
    return infoset1

In [10]:
# Extracts the following in order: k_factor, rating_system, tournament_type, time_control
def extract_tournament_info2(soup: BeautifulSoup) -> List[str]:
    upper_table: ResultSet[Tag] = soup.find_all('table', attrs={
            'border': '0',
            'bgcolor': 'FFFFFF',
            'cellpadding': '3',
            'cellspacing': '0'
        })
    header_box: Tag = upper_table[1]
    rules: str = header_box.find_all('b')[3].text
    rules_list: List[str] = rules.split(' ')
    # A small number of uscf pages break with the formatting conventions used by this scrape.
    # In this case, raise an exception.
    if len(rules_list) < 5:
        raise Exception('extract_tournament_info2() scraped fewer elements than expected.')

    infoset2: List[str] = []
    infoset2.append(rules_list[7])
    infoset2.append(rules_list[11])
    infoset2.append(rules_list[16])
    infoset2.append(rules_list[20])
    return infoset2


In [11]:
import re
from re import Match

# Includes both player names and links. Each link needs to be
# preceded by https://www.uschess.org/msa/.
def get_player_names_and_urls(soup: BeautifulSoup) -> ResultSet[Tag]:
    results: ResultSet[Tag] = soup.find_all('a', {'href': lambda x: x and x.startswith('MbrDtlMain')})
    players: List[Tuple[str]] = []
    for result in results:
        players.append((result.text, 'https://www.uschess.org/msa/' + result['href']))
    return players

def extract_tabular_results(soup: BeautifulSoup) -> List[str]:
    pre_results: ResultSet[Tag] = soup.find_all('pre')
    # Using .stripped_strings returns the data not surrounded by an HTML tag, which is the
    # results data we want.
    results_raw: Generator = pre_results[0].stripped_strings
    results_clean: List[Tuple[str]] = []
    results_query: str = r'\|\d+\.\d+\s*(?:\|[A-Z]\s*\d*)+'
    state_query: str = r'\|\s+[A-Z][A-Z]\s+\|'
    rating_query: str = r'(\|\s*\d+\s*\/\s*[A-Z]+:[A-Za-z\s0-9]+->[A-Za-z\s0-9]+)([\sBW\|]*)'

    player_rows: List[Tuple[str]] = []
    for string in results_raw:
        score: List[str] = re.findall(results_query, string)
        state: List[str] = re.findall(state_query, string)
        ratings_and_pairings = re.findall(rating_query, string)

        ratings: str = ''
        pairings: str = ''
        if not score:
            score: str = ''
        if not state:
            state: str = ''
        if ratings_and_pairings:
            ratings = ratings_and_pairings[0][0]
            pairings = ratings_and_pairings[0][1]

        if score or state or ratings or pairings:
            player_rows.append((score, state, ratings, pairings))

    clean_tabs: List[List[str]] = []
    for row in player_rows:
        player_data: List[str] = []
        # Player scores in different rounds
        if row[0]:
            split_scores: List[str] = row[0][0].split('|')
            rounds: List[str] = []
            for s in split_scores[1:]:
                s = s.strip()
                if s != '' and s != ',':
                    rounds.append(s)
            player_data.append(rounds)
        
        # State codes
        if type(row[1]) == list:
            player_data.append(row[1][0].strip('|').strip('\n').strip(' '))
        else:
            player_data.append('')

        # Rating changes during the tournament
        if row[2]:
            split_ratings: List[str] = row[2].strip('|').split('/')
            player_data.append(split_ratings[0].strip())
            second_split: List[str] = split_ratings[-1].split(':')
            player_data.append(second_split[0].strip())
            before_rating, after_rating = second_split[1].split('->')
            player_data.append(before_rating.strip())
            player_data.append(after_rating.strip())
        else:
            player_data.append('')
            player_data.append('')
            player_data.append('')
            player_data.append('')

        if row[3]:
            player_data.append(row[3].strip())
        else:
            player_data.append('')

        clean_tabs.append(player_data)
    return clean_tabs

In [12]:
def get_max_id(conn: Connection) -> int:
    cursor: Cursor = conn.cursor()
    cursor.execute("SELECT MAX(rowid) FROM uscf_urls")
    return cursor.fetchall()[0][0]

In [13]:
# Returns an int representing the tournament's primary key
def store_tournament(conn: Connection, metadata1: List[str], metadata2: List[str], url_fk: int) -> int:
    cur: Cursor = conn.cursor()
    input_list: List[str] = []
    input_list.extend(metadata1)
    input_list.extend(metadata2)
    input_list.append(url_fk)
    cur.execute("""INSERT INTO uscf_tournaments (
                tournament_name, tournament_code, 
                event_date, received_date, entered_date, 
                rated_date, section_count, player_count,
                k_factor, rating_system, tournament_type,
                time_control, urls_id
                ) VALUES (?, ?, ?, ?, ?, ?, ?,
                ?, ?, ?, ?, ?, ?)
    """, tuple(input_list))
    conn.commit()
    cur.execute("SELECT MAX(rowid) FROM uscf_tournaments")
    return cur.fetchall()[0][0]

# Returns a List[int] representing the primary keys for each player observation
def store_players(conn: Connection, players: List[Tuple[str]], tournament_results: List, tournament_fk: int) -> List[int]:
    cur: Cursor = conn.cursor()
    if len(players) != len(tournament_results):
        raise Exception('players and tournament_results should be the same shape')
    
    max_ids: List[int] = []
    for i in range(len(players)):
        player: str = players[i]
        result: str = tournament_results[i]
        name: str = player[0]
        seed_number: str = str(i + 1)
        url: str = player[1]
        uscf_id: str = result[2]
        overall_record: str = result[0][:1][0]
        state_code: str = result[1]
        rating_type: str = result[3]
        before_rating: str = result[4]
        after_rating: str = result[5]
        color_assignments: str = result[6]
        input_tuple: Tuple[str] = tuple([name, seed_number, url, uscf_id,
                                overall_record, state_code, rating_type, 
                                before_rating, after_rating, color_assignments, tournament_fk])
        cur.execute("""INSERT INTO uscf_player_observations (name, seed_number, 
            url, uscf_id, record, state_code, rating_type, 
            before_rating, after_rating, color_assignments, uscf_tournaments_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?
            )""", input_tuple)
        conn.commit()
        cur.execute("SELECT MAX(rowid) FROM uscf_player_observations")
        max_ids.append(cur.fetchall()[0][0])
    return max_ids

def store_rounds(conn: Connection, tournament_fk: int, player_fks: List[int], tournament_results: List[str]) -> None:
    cur: Cursor = conn.cursor()
    for j in range(len(tournament_results)):
        round_results: List[str] = tournament_results[j][0][1:]
        player_fk: int = player_fks[j]
        for i in range(len(round_results)):
            round_number: int = i + 1

            result: Match = re.findall(r'^[A-Za-z]', round_results[i])
            opponent_seed: Match = re.findall(r'[0-9]+$', round_results[i])
            if result:
                result: str = result[0]
            else:
                result: str = None
            if opponent_seed:
                opponent_seed: str = opponent_seed[0]
            else:
                opponent_seed: str = None

            cur.execute("""INSERT INTO uscf_rounds (
                round_number, result, opponent, 
                uscf_tournaments_id, uscf_player_id
            ) VALUES (
                ?, ?, ?, ?, ?
            )""", (round_number, result, opponent_seed, tournament_fk, player_fk))
            conn.commit()

In [14]:
# conn: Connection = sqlite3.Connection('scrape_data.db')
# max_id: int = get_max_id(conn)

# exception_count = 0
# for url_fk in range(1, 1 + max_id):
#     print('Processing:', url_fk)
#     soup: BeautifulSoup = get_tournament_page(url_fk)

#     try:
#         # Extract text data from soup object
#         metadata1: List[str] = extract_tournament_info1(soup)
#         metadata2: List[str] = extract_tournament_info2(soup)
#         players: List[Tuple[str]] = get_player_names_and_urls(soup)
#         tournament_results: List = extract_tabular_results(soup)
#         # Store data in db
#         tournament_fk: int = store_tournament(conn, metadata1, metadata2, url_fk)
#         player_fks: List[int] = store_players(conn, players, tournament_results, tournament_fk)
#         store_rounds(conn, tournament_fk, player_fks, tournament_results)
#     except Exception as e:
#         print(e)
#         exception_count += 1

# print('Number of exceptions', exception_count)

In [15]:
# Executed this code in order to fix the database schema

# conn: Connection = sqlite3.Connection('scrape_data.db')
# add_provisional_column: str = """ALTER TABLE uscf_player_observations
#                                 ADD COLUMN provisional_count INTEGER"""
# cur: Cursor = conn.cursor()
# cur.execute(add_provisional_column)

In [16]:
# Ran this cell to store provisional count and clean up before/after_ratings

# conn: Connection = sqlite3.Connection('scrape_data.db')
# cur: Cursor = conn.cursor()

# add_provisional_counts: str = """
#     UPDATE uscf_player_observations
#     SET provisional_count = SUBSTR(after_rating, INSTR(after_rating, 'P') + 1, LENGTH(after_rating))
#     WHERE after_rating LIKE '%P%'
#     """
# cur.execute(add_provisional_counts)
# conn.commit()

# clean_after_rating: str = """
#     UPDATE uscf_player_observations
#     SET after_rating = SUBSTR(after_rating, 1, INSTR(after_rating, 'P') - 1)
#     WHERE after_rating LIKE '%P%'
# """
# cur.execute(clean_after_rating)
# conn.commit()

# clean_before_rating: str = """
#     UPDATE uscf_player_observations
#     SET before_rating = SUBSTR(before_rating, 1, INSTR(before_rating, 'P') - 1)
#     WHERE before_rating LIKE '%P%'
# """
# cur.execute(clean_before_rating)
# conn.commit()

In [17]:
# Add columns to represent data on players own pages
# conn: Connection = sqlite3.Connection('scrape_data.db')
# query_list: List[str] = []

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN regular_elo INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN quick_elo INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN blitz_elo INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN online_regular_elo INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN online_quick_elo INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN online_blitz_elo INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN regular_date_rated TEXT;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN quick_date_rated TEXT;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN blitz_date_rated TEXT;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN online_regular_date_rated TEXT;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN online_quick_date_rated TEXT;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN online_blitz_date_rated TEXT;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN regular_provisional_num INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN quick_provisional_num INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN blitz_provisional_num INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN online_regular_provisional_num INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN online_quick_provisional_num INTEGER;""")

# query_list.append("""ALTER TABLE uscf_player_observations
# ADD COLUMN online_blitz_provisional_num INTEGER;""")

# cur: Cursor = conn.cursor()
# for query in query_list:
#     cur.execute(query)
#     conn.commit()

In [18]:
# Returns a list of two-tuples. Each two-tuple contains the player's USCF id
# and their unique url.
def get_player_ids_urls() -> List[Tuple[str]]:
    cur: Cursor = sqlite3.Connection('scrape_data.db').cursor()
    cur.execute("""SELECT uscf_id, url
                    FROM uscf_player_observations
    """)
    return cur.fetchall()

def get_player_page(url: str) -> str:
    request: Response = requests.get(url)
    print(url)
    print('Status Code:', request.status_code)
    return BeautifulSoup(request.text, 'html.parser').text

In [19]:
# Returns two lists, representing ordinary ratings and provisional ratings respectively. 
# Each list has tuples of these lengths:
# BASE: 3
# PROVISIONAL: 4
def match_player_data(page: str) -> List[Tuple[str]]:
    BASE_QUERY: str = '([a-zA-Z-]+\s[a-zA-Z]+)\s+(\d{3,4})\xA0*\s*(\d{4}-\d{2})'
    UNRATED_QUERY: str = '([a-zA-Z]*\s[a-zA-Z-]*\s[a-zA-Z]*)\s+(\(Unrated\))'
    PROVISIONAL_QUERY: str = '([a-zA-Z]*\s[a-zA-Z-]*\s[a-zA-Z]*)\s+(\d{3,4})\s\(Based\son\s(\d{1,3})\sgames\)\xA0*\s*(\d{4}-\d{2})'
    base_results: List[Tuple[str]] = re.findall(BASE_QUERY, page)
    provisional_results: List[Tuple[str]] = re.findall(PROVISIONAL_QUERY, page)
    return base_results, provisional_results

In [20]:
# Inserts all non-provisional ratings for one player.
def insert_normal_ratings(conn: Connection, ratings: List[Tuple[str]], uscf_id: int) -> None:
    cur: Cursor = conn.cursor()
    # List of values to insert of length 12 with this structure:
    # [6 rating types, 6 rating dates]
    db_insert_list: List[str] = [None for x in range(12)]
    for rating in ratings:
        rating_type: str = rating[0].strip()
        elo: str = rating[1].strip()
        date: str = rating[2].strip()
        if rating_type == 'Regular Rating':
            db_insert_list[0] = elo
            db_insert_list[1] = date
        elif rating_type == 'Quick Rating':
            db_insert_list[2] = elo
            db_insert_list[3] = date
        elif rating_type == 'Blitz Rating':
            db_insert_list[4] = elo
            db_insert_list[5] = date
        elif rating_type == 'Online-Regular Rating':
            db_insert_list[6] = elo
            db_insert_list[7] = date
        elif rating_type == 'Online-Quick Rating':
            db_insert_list[8] = elo
            db_insert_list[9] = date
        elif rating_type == 'Online-Blitz Rating':
            db_insert_list[10] = elo
            db_insert_list[11] = date
    db_insert_list.append(uscf_id)
    print(db_insert_list)
    cur.execute("""UPDATE uscf_player_observations
        SET regular_elo = COALESCE(?, regular_elo), 
            regular_date_rated = COALESCE(?, regular_date_rated), 
            quick_elo = COALESCE(?, quick_elo), 
            quick_date_rated = COALESCE(?, quick_date_rated), 
            blitz_elo = COALESCE(?, blitz_elo), 
            blitz_date_rated = COALESCE(?, blitz_date_rated), 
            online_regular_elo = COALESCE(?, online_regular_elo), 
            online_regular_date_rated = COALESCE(?, online_regular_date_rated), 
            online_quick_elo = COALESCE(?, online_quick_elo), 
            online_quick_date_rated = COALESCE(?, online_quick_date_rated), 
            online_blitz_elo = COALESCE(?, online_blitz_elo), 
            online_blitz_date_rated = COALESCE(?, online_blitz_date_rated)
        WHERE uscf_id = ?""", tuple(db_insert_list))
    conn.commit()

# Inserts all provisional ratings for one player.
def insert_provisional_ratings(conn: Connection, ratings: List[Tuple[str]], uscf_id: int) -> None:
        cur: Cursor = conn.cursor()
        # List of values to insert of length 18 with this structure:
        # [6 rating types, 6 rating dates, 6 provisional counts]
        db_insert_list: List[str] = [None for x in range(18)]
        for rating in ratings:
            rating_type: str = rating[0].strip()
            elo: str = rating[1].strip()
            provisional_games: str = rating[2].strip()
            date: str = rating[3].strip()
            if rating_type == 'Regular Rating':
                db_insert_list[0] = elo
                db_insert_list[1] = date
                db_insert_list[2] = provisional_games
            elif rating_type == 'Quick Rating':
                db_insert_list[3] = elo
                db_insert_list[4] = date
                db_insert_list[5] = provisional_games
            elif rating_type == 'Blitz Rating':
                db_insert_list[6] = elo
                db_insert_list[7] = date
                db_insert_list[8] = provisional_games
            elif rating_type == 'Online-Regular Rating':
                db_insert_list[9] = elo
                db_insert_list[10] = date
                db_insert_list[11] = provisional_games
            elif rating_type == 'Online-Quick Rating':
                db_insert_list[12] = elo
                db_insert_list[13] = date
                db_insert_list[14] = provisional_games
            elif rating_type == 'Online-Blitz Rating':
                db_insert_list[15] = elo
                db_insert_list[16] = date
                db_insert_list[17] = provisional_games
        db_insert_list.append(uscf_id)
        print(db_insert_list)
        cur.execute("""UPDATE uscf_player_observations
        SET regular_elo = COALESCE(?, regular_elo), 
            regular_date_rated = COALESCE(?, regular_date_rated), 
            regular_provisional_num = COALESCE(?, regular_provisional_num), 
            quick_elo = COALESCE(?, quick_elo), 
            quick_date_rated = COALESCE(?, quick_date_rated), 
            quick_provisional_num = COALESCE(?, quick_provisional_num), 
            blitz_elo = COALESCE(?, blitz_elo), 
            blitz_date_rated = COALESCE(?, blitz_date_rated), 
            blitz_provisional_num = COALESCE(?, blitz_provisional_num),
            online_regular_elo = COALESCE(?, online_regular_elo), 
            online_regular_date_rated = COALESCE(?, online_regular_date_rated), 
            online_regular_provisional_num = COALESCE(?, online_regular_provisional_num), 
            online_quick_elo = COALESCE(?, online_quick_elo), 
            online_quick_date_rated = COALESCE(?, online_quick_date_rated), 
            online_quick_provisional_num = COALESCE(?, online_quick_provisional_num), 
            online_blitz_elo = COALESCE(?, online_blitz_elo), 
            online_blitz_date_rated = COALESCE(?, online_blitz_date_rated), 
            online_blitz_provisional_num = COALESCE(?, online_blitz_provisional_num)
        WHERE uscf_id = ?""", tuple(db_insert_list))
        conn.commit()

In [21]:
def download_player_pages() -> None:
    ids_and_urls: List[Tuple[str]] = get_player_ids_urls()
    for player_tuple in ids_and_urls:
        uscf_id = int(player_tuple[0])
        url = player_tuple[1]

        page: str = get_player_page(url)
        normal, provisional = match_player_data(page)

        conn: Connection = sqlite3.Connection('scrape_data.db')
        if normal:
            insert_normal_ratings(conn, normal, uscf_id)
        if provisional:
            insert_provisional_ratings(conn, provisional, uscf_id)

download_player_pages()

https://www.uschess.org/msa/MbrDtlMain.php?16777221
Status Code: 200
[('Online-Quick Rating', '1400', '2023-02'), ('Online-Blitz Rating', '1458', '2023-02')]
[None, None, None, None, None, None, None, None, '1400', '2023-02', '1458', '2023-02', 16777221]
[('\nRegular Rating', '1425', '10', '2023-02'), ('\nOnline-Regular Rating', '1315', '22', '2023-02')]
['1425', '2023-02', '10', None, None, None, None, None, None, '1315', '2023-02', '22', None, None, None, None, None, None, 16777221]


In [22]:
# DROP BAD TUPLES LATER!

conn: Connection = sqlite3.Connection('scrape_data.db')
cur: Cursor = conn.cursor()
cur.execute("SELECT * FROM uscf_player_observations WHERE uscf_id = 16777221")
cur.fetchall()

[(309,
  'PANDEESWARAN BHOOPATHY',
  50,
  'https://www.uschess.org/msa/MbrDtlMain.php?16777221',
  16777221,
  '1.0',
  'CA',
  'OQ',
  '1400',
  '1407',
  '|     |B    |B    |W    |     |     |',
  7,
  None,
  1425,
  None,
  None,
  1315,
  1400,
  1458,
  '2023-02',
  None,
  None,
  '2023-02',
  '2023-02',
  '2023-02',
  10,
  None,
  None,
  22,
  None,
  None),
 (443,
  'PANDEESWARAN BHOOPATHY',
  24,
  'https://www.uschess.org/msa/MbrDtlMain.php?16777221',
  16777221,
  '2.5',
  'CA',
  'OB',
  '1458',
  '1535',
  '|     |B    |     |W    |B    |W    |',
  11,
  None,
  1425,
  None,
  None,
  1315,
  1400,
  1458,
  '2023-02',
  None,
  None,
  '2023-02',
  '2023-02',
  '2023-02',
  10,
  None,
  None,
  22,
  None,
  None),
 (544,
  'PANDEESWARAN BHOOPATHY',
  1,
  'https://www.uschess.org/msa/MbrDtlMain.php?16777221',
  16777221,
  '6.0',
  'CA',
  'OB',
  '1535',
  '1552',
  '|     |B    |W    |B    |B    |W    |W    |B    |',
  14,
  None,
  1425,
  None,
  None,
  1315,
